In [1]:
include("setup.jl")

┌ Info: Installing packages. This may take up to a few minutes.
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:1


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Info: Loading packages
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:7
┌ Info: All packages successfully imported.
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:21


# Asymmetric Travelling Salesman Problem

**Originally Contributed by**: Mathieu Tanneau and Alexis Montoison

https://www.movable-type.co.uk/scripts/latlong.html
### Problem description

$$
    \begin{array}{cl}
        \min_{x} \ \ \ &
            \sum_{i, j} d_{i, j} x_{i, j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in \Omega\\
        &    \sum_{i} x_{i, j} = 1, \ \ \ \forall j \in \Omega\\
        &    x_{k,k} = 0 \ \ \ \forall k \in \Omega\\
        &   \sum_{i} \sum_{j \ne i} x_{i,j} \le |S| - 1, \forall S \subset \Omega, 2 \le |S| \le N-1\\
        &   x_{i, j} \in \{0, 1\}, \ \ \ \forall (i,j) \in \Omega^2 , i \ne j\\
        & \Omega = \{1, \dots, N\}
    \end{array}
$$

In [2]:
d = readdlm("./tsp/distances_quebec.txt")

9×9 Array{Float64,2}:
   0.0  256.0  137.0  152.0   193.0  461.0   542.0   627.0   889.0
 256.0    0.0  125.0  209.0   422.0  206.0   313.0   848.0   719.0
 137.0  125.0    0.0  143.0   297.0  326.0   424.0   723.0   829.0
 152.0  209.0  143.0    0.0   337.0  414.0   484.0   776.0   737.0
 193.0  422.0  297.0  337.0     0.0  623.0   720.0   517.0  1075.0
 461.0  206.0  326.0  414.0   623.0    0.0   249.0   840.0   700.0
 542.0  313.0  424.0  484.0   720.0  249.0     0.0  1089.0   511.0
 627.0  848.0  723.0  776.0   517.0  840.0  1089.0     0.0  1552.0
 889.0  719.0  829.0  737.0  1075.0  700.0   511.0  1552.0     0.0

In [3]:
c = readdlm("./tsp/cities_quebec.txt")

9×1 Array{Any,2}:
 "Montréal"      
 "Québec"        
 "Trois-Rivières"
 "Sherbrooke"    
 "Hull"          
 "Chicoutimi"    
 "Rimouski"      
 "Rouyn"         
 "Moncton"       

In [4]:
"""
    all_subsets(x)

Compute all subsets of elements of vector `x`.
"""
function all_subsets(x::Vector{T}) where T
    res = Vector{T}[[]]  # Vector of vectors
    for elem in x, j in eachindex(res)
        push!(res, [res[j] ; elem])
    end
    return res
end

all_subsets

In [5]:
all_subsets([1:5;])

32-element Array{Array{Int64,1},1}:
 []             
 [1]            
 [2]            
 [1, 2]         
 [3]            
 [1, 3]         
 [2, 3]         
 [1, 2, 3]      
 [4]            
 [1, 4]         
 [2, 4]         
 [1, 2, 4]      
 [3, 4]         
 ⋮              
 [3, 5]         
 [1, 3, 5]      
 [2, 3, 5]      
 [1, 2, 3, 5]   
 [4, 5]         
 [1, 4, 5]      
 [2, 4, 5]      
 [1, 2, 4, 5]   
 [3, 4, 5]      
 [1, 3, 4, 5]   
 [2, 3, 4, 5]   
 [1, 2, 3, 4, 5]

In [6]:
"""
    solve_tsp(D, optimizer)

Compute a shortest TSP tour given then matrix distance `D`.
"""
function solve_tsp(D, optimizer)
    # Number of cities
    N = size(D, 1)
    N == size(D, 2) || throw(DimensionMismatch())                     # sanity check: `D` is square
    N <= 16 || error("N cannot be larger than 16 for memory safety")  # sanity check: `N` is not too large
    
    # Instantiate a model
    mip = Model(with_optimizer(optimizer))

    # I. Create arc variables
    @variable(mip, X[1:N, 1:N], Bin)

    # II. Set objective
    @objective(mip, Min, sum(X.*D)) # sum(D[i,j] * X[i,j] for i=1:N, j=1:N)

    # III. Add constraints to the model
    
    # III.1 
    # No city can be its own follower in the tour
    for k in 1:N
        @constraint(mip, X[k,k] == 0.0)
    end

    # III.2
    # Each city has one predecessor and one successor
    for i in 1:N
        @constraint(mip, sum(X[i, j] for j in 1:N) == 1.0)
        @constraint(mip, sum(X[j, i] for j in 1:N) == 1.0)
    end
    
    # III.3
    # We only want a cycle of length N
    tours = all_subsets([1:N;])
    for st in tours
        T = length(st)
        # Sub-tour elimination constraints
        if 2 ≤ T ≤ N-1
            @constraint(mip, sum(X[st[k],st[k+1]] for k=1:T-1) + X[st[T],st[1]] ≤ T-1)
            @constraint(mip, sum(X[st[k+1],st[k]] for k=1:T-1) + X[st[1],st[T]] ≤ T-1)
        end
    end
    
    # Solve MIP model
    optimize!(mip)
    
    println("Optimal tour length is ", objective_value(mip))

    # Return solution
    return value.(X)
end

solve_tsp

In [7]:
solve_tsp(d, Cbc.Optimizer)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3274 - 0.00 seconds
Cgl0002I 9 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 930 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 762 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 583 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 150 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 13 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 984 rows, 72 columns (72 integer (72 of which binary)) and 7083 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 19 integers unsatisfied sum - 6.33333
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 3901 iterations 35
Cbc0038I Solution found of 3901
Cbc0038I Befor

9×9 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [8]:
sol = solve_tsp(d, GLPK.Optimizer)

Optimal tour length is 3413.0


9×9 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [9]:
include("tsp/RoadTrip.jl")

RoadTrip (generic function with 1 method)

In [10]:
RoadTrip(9, c, sol)

Montréal
Sherbrooke
Moncton
Rimouski
Chicoutimi
Québec
Trois-Rivières
Rouyn
Hull


![](tsp/solution_quebec.png)